#### Import modules, declare constants, export data

In [1]:
from bqWrapper.bq import bqWrapper
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import seaborn as sns, matplotlib.pyplot as plt
import numpy as np

/opt/conda/anaconda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
bqw = bqWrapper()
spark = bqw.connection
df_train = bqw.create_bigquery_connection(connection=spark, table='training_data')
df_val = bqw.create_bigquery_connection(connection=spark, table='validation_data')

In [3]:
COHORTS_DIMENSIONS = ['first_touch_date', 'traffic_source', 'os', 'country']
TARGET_VAR = 'cohort_ltv_avg_lifetime'
evaluator = RegressionEvaluator(labelCol=TARGET_VAR, predictionCol='prediction', metricName='rmse')

#### Generate train, test data

Here we use Min-Max scaling. Since our data is sparse, normalizing values to 0-1 scale will help adjust for this factor

In [4]:
# Min-Max scaled data
feature_list = [col for col in df_train.columns if (col not in COHORTS_DIMENSIONS) and (col != TARGET_VAR)]
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in feature_list]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in feature_list]
pipeline = Pipeline(stages=assemblers + scalers)
scalerModel = pipeline.fit(df_train)
df_train_trans = scalerModel.transform(df_train)
assembler = VectorAssembler(inputCols=feature_list, outputCol='features')
df_train_trans = assembler.transform(df_train_trans)
(X_train_mm, X_test_mm) = df_train_trans.randomSplit([0.8, 0.2], seed=143)
# Validation set
df_val_trans = assembler.transform(df_val)
df_val_pipe = df_val.drop(*(COHORTS_DIMENSIONS+['features']))
# Adjusted for cv
X_train_pipe = X_train_mm.drop(*(COHORTS_DIMENSIONS+['features']))
X_test_pipe = X_test_mm.drop(*(COHORTS_DIMENSIONS+['features']))

#### Dummy baseline average-based model

In [5]:
target_avg = X_train_mm.select(mean(df_train.cohort_ltv_avg_lifetime)).collect()[0][f'avg({TARGET_VAR})']
df_avg = X_test_mm.select(TARGET_VAR)
df_avg = df_avg.withColumn('prediction', lit(target_avg))

In [6]:
avg_rmse = evaluator.evaluate(df_avg)
avg_rmse

9.992390089910161

In [9]:
df_avg_val = df_val.select(TARGET_VAR)
df_avg_val = df_avg_val.withColumn('prediction', lit(target_avg))
avg_rmse_val = evaluator.evaluate(df_avg_val)
avg_rmse_val

5.015719873031029

#### Linear Regression with Regularization

In [11]:
# Default parameters for all types of regression
FEATURES_COL = 'features'
STANDARDIZATION = False
MAXITER = 10000
REGPARAM = 1

##### Lasso

In [6]:
lasso = LinearRegression(
    labelCol=TARGET_VAR,
    featuresCol=FEATURES_COL,
    elasticNetParam=1,
    regParam=REGPARAM,
    standardization=STANDARDIZATION,
    maxIter=MAXITER
)

In [10]:
lasso_model = lasso.fit(X_train_mm)
lasso_predictions = lasso_model.transform(X_test_mm)

In [11]:
lasso_rmse = evaluator.evaluate(lasso_predictions)
lasso_rmse

8.374706262242034

Barely better then baseline. Implement cross-validation to improve results

In [12]:
pipeline = Pipeline(stages=[assembler, lasso])
paramGrid_lasso = ParamGridBuilder() \
    .addGrid(lasso.regParam, [0.1, 0.5, 1, 5, 10]) \
    .build()
crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid_lasso,
    evaluator=evaluator,
    numFolds=10
)

In [13]:
lasso_cv_model = crossval.fit(X_train_pipe)
lasso_cv_predictions = lasso_cv_model.transform(X_test_pipe)

In [14]:
lasso_cv_rmse = evaluator.evaluate(lasso_cv_predictions)
lasso_cv_rmse

8.404330442625282

In [15]:
for i in zip(paramGrid_lasso, lasso_cv_model.avgMetrics):
    print(i)

({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 0.1}, 6.884123118104254)
({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 0.5}, 6.6098746101420085)
({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 1.0}, 6.683507159372043)
({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 5.0}, 7.02706676288023)
({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 10.0}, 7.336225221398381)


Cross-validation didn't really help much, although its train RMSE was at least quite an improvement. Let us evaluate both models on validation set

In [17]:
lasso_predictions_val = lasso_model.transform(df_val_trans)
lasso_rmse_val = evaluator.evaluate(lasso_predictions_val)
lasso_rmse_val

3.817859626746012

In [20]:
lasso_cv_predictions_val = lasso_cv_model.transform(df_val_pipe)
lasso_cv_rmse_val = evaluator.evaluate(lasso_cv_predictions_val)
lasso_cv_rmse_val

3.8835436293697185

<p>Lasso model on default parameters outperformed the CV one on validation set as well</p>
<p>Let us test other models before trying to extract some evidence from our predictions</p>

##### Ridge

In [22]:
ridge = LinearRegression(
    labelCol=TARGET_VAR,
    featuresCol=FEATURES_COL,
    elasticNetParam=0,
    regParam=REGPARAM,
    standardization=STANDARDIZATION,
    maxIter=MAXITER
)

In [23]:
ridge_model = ridge.fit(X_train_mm)
ridge_predictions = ridge_model.transform(X_test_mm)

In [24]:
ridge_rmse = evaluator.evaluate(ridge_predictions)
ridge_rmse

8.493504900099856

The result is worse than the base Lasso model. Let us try to improve it with CV

In [25]:
pipeline = Pipeline(stages=[assembler, ridge])
paramGrid_ridge = ParamGridBuilder() \
    .addGrid(ridge.regParam, [0.1, 0.5, 1, 5, 10]) \
    .build()
crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid_ridge,
    evaluator=evaluator,
    numFolds=10
)

In [26]:
ridge_cv_model = crossval.fit(X_train_pipe)
ridge_cv_predictions = ridge_cv_model.transform(X_test_pipe)

In [27]:
ridge_cv_rmse = evaluator.evaluate(ridge_cv_predictions)
ridge_cv_rmse

8.435716567217392

In [28]:
for i in zip(paramGrid_ridge, ridge_cv_model.avgMetrics):
    print(i)

({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 0.1}, 7.041097972345739)
({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 0.5}, 6.862878757818502)
({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 1.0}, 6.783176302305053)
({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 5.0}, 6.670064829309338)
({Param(parent='LinearRegression_4514dbe62b5b', name='regParam', doc='regularization parameter (>= 0).'): 10.0}, 6.648940445097065)


<p>CV slightly improved the results of base model for Ridge regression, and it is still worse than base</p>

<p>However, we can go into second round of CV as we didn't reach the turning point</p>

In [29]:
pipeline = Pipeline(stages=[assembler, ridge])
paramGrid_ridge = ParamGridBuilder() \
    .addGrid(ridge.regParam, [i for i in range(10, 35, 5)]) \
    .build()
crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid_ridge,
    evaluator=evaluator,
    numFolds=10
)

In [30]:
ridge_cv_model = crossval.fit(X_train_pipe)
ridge_cv_predictions = ridge_cv_model.transform(X_test_pipe)

In [31]:
ridge_cv_rmse = evaluator.evaluate(ridge_cv_predictions)
ridge_cv_rmse

8.412693361104662

In [33]:
for i in zip(paramGrid_ridge, ridge_cv_model.avgMetrics):
    print(i)

({Param(parent='LinearRegression_379bee5804b7', name='regParam', doc='regularization parameter (>= 0).'): 10.0}, 6.648940445097065)
({Param(parent='LinearRegression_379bee5804b7', name='regParam', doc='regularization parameter (>= 0).'): 15.0}, 6.639043413764755)
({Param(parent='LinearRegression_379bee5804b7', name='regParam', doc='regularization parameter (>= 0).'): 20.0}, 6.636158074246821)
({Param(parent='LinearRegression_379bee5804b7', name='regParam', doc='regularization parameter (>= 0).'): 25.0}, 6.627298251874045)
({Param(parent='LinearRegression_379bee5804b7', name='regParam', doc='regularization parameter (>= 0).'): 30.0}, 6.635134794786982)


We managed to achieve a turning point at `regParam=25`<br><br>
Still, test RMSE for CV Ridge model is worse than for base Lasso. Let us evaluate on validation set

In [34]:
ridge_predictions_val = ridge_model.transform(df_val_trans)
ridge_rmse_val = evaluator.evaluate(ridge_predictions_val)
ridge_rmse_val

3.9214690700340626

In [35]:
ridge_cv_predictions_val = ridge_cv_model.transform(df_val_pipe)
ridge_cv_rmse_val = evaluator.evaluate(ridge_cv_predictions_val)
ridge_cv_rmse_val

3.8701872186442867

Ridge model didn't outperform Lasso. Let us try the ElasticNet model, which combines both types of regularization, to see if it could outperform base Lasso model

#### ElasticNet

In [12]:
elastic = LinearRegression(
    labelCol=TARGET_VAR,
    featuresCol=FEATURES_COL,
    elasticNetParam=0.5,
    regParam=REGPARAM,
    standardization=STANDARDIZATION,
    maxIter=MAXITER
)

In [28]:
elastic_model = elastic.fit(X_train_mm)
elastic_predictions = elastic_model.transform(X_test_mm)

In [38]:
elastic_rmse = evaluator.evaluate(elastic_predictions)
elastic_rmse

8.405489637438942

First try is worse than base Lasso. However, ElasticNet has two parameters to tune, so here we expect CV to bring much better improvement

In [13]:
pipeline = Pipeline(stages=[assembler, elastic])
paramGrid_elastic = ParamGridBuilder() \
    .addGrid(elastic.regParam, [0.1, 0.5, 1, 5, 10]) \
    .addGrid(elastic.elasticNetParam, np.arange(0.1, 1, 0.2)) \
    .build()
crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid_elastic,
    evaluator=evaluator,
    numFolds=10
)

In [14]:
elastic_cv_model = crossval.fit(X_train_pipe)
elastic_cv_predictions = elastic_cv_model.transform(X_test_pipe)

In [15]:
elastic_cv_rmse = evaluator.evaluate(elastic_cv_predictions)
elastic_cv_rmse

8.40894600272515

In [22]:
scores = [val for val in elastic_cv_model.avgMetrics]
min_score = np.min(scores)
min_score_idx = scores.index(np.min(scores))
for i in range(min_score_idx-1, min_score_idx+2):
    print(paramGrid_elastic[i], elastic_cv_model.avgMetrics[i])

{Param(parent='LinearRegression_981ce767d4dc', name='regParam', doc='regularization parameter (>= 0).'): 1.0, Param(parent='LinearRegression_981ce767d4dc', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.9000000000000001} 7.96457479684676
{Param(parent='LinearRegression_981ce767d4dc', name='regParam', doc='regularization parameter (>= 0).'): 5.0, Param(parent='LinearRegression_981ce767d4dc', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1} 7.954137218939503
{Param(parent='LinearRegression_981ce767d4dc', name='regParam', doc='regularization parameter (>= 0).'): 5.0, Param(parent='LinearRegression_981ce767d4dc', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is a

CV ElasticNet model is still worse than base Lasso. However, it is worth mentioning that this model didn't really overfit on training data, so it is more robust one, and perhaps will provide better results on validation set

In [29]:
elastic_predictions_val = elastic_model.transform(df_val_trans)
elastic_rmse_val = evaluator.evaluate(elastic_predictions_val)
elastic_rmse_val

3.880779601994567

In [30]:
elastic_cv_predictions_val = elastic_cv_model.transform(df_val_pipe)
elastic_cv_rmse_val = evaluator.evaluate(elastic_cv_predictions_val)
elastic_cv_rmse_val

3.8643459553038966

Still, it is worse than base Lasso. Let us try to further improve Lasso model by trying another scaling algorithm, which in this case will be log-scaling